In [45]:
import json
import math
import escher
import cobra
import cobrakbase
import modelseed_escher
from cobrakbase.core import KBaseFBAModel
from modelseed_escher.core import EscherMap

In [7]:
%run ../lib/kb_escher/kb_escher_utils.py

In [1]:
params = {
    'workspace_id': 37534, 
    'column_size': '4', 
    'report_height': '800', 
    'workspace_name': 'filipeliu:narrative_1580723870549', 
    'grid_maps': [
        {
            'object_ids': ['37534/91/1'], 
            'map_id': 'custom', 
            'user_map_id': '37534/38/1', 
            'model_id': '37534/90/3'
        }, 
        {
            'object_ids': ['37534/92/1'], 
            'map_id': 'FBA Checked TCA Cycle', 
            'user_map_id': '37534/88/1', 
            'model_id': '37534/89/1'
        }
    ]
}

In [35]:
token = "64XQ7SABQILQWSEW3CQKZXJA63DXZBGH"
api = cobrakbase.KBaseAPI(token, dev=True)

In [21]:
escher_seed = modelseed_escher.EscherManager(escher)

In [57]:
column_size = int(params['column_size'])
grid_cells = len(params['grid_maps'])
grid_size = (column_size, math.ceil(grid_cells / column_size))
grid_size

(4, 1)

In [56]:
def build_grid_cell(grid_cell, escher_seed, api, alias = None):
    em = None
    if not grid_cell['map_id'] == 'custom':
        em = escher_seed.get_map('ModelSEED', 'ModelSEED', grid_cell['map_id'])
    else:
        ref = api.get_object_info_from_ref(grid_cell['user_map_id'])
        map_data = api.get_object(ref.id, ref.workspace_id)
        em = modelseed_escher.core.EscherMap([map_data['metadata'], map_data['layout']])

    cmp_id = 'c0' #should come from parameters
    ref = api.get_object_info_from_ref(grid_cell['model_id'])
    fbamodel = KBaseFBAModel(api.get_object(ref.id, ref.workspace_id))
    if 'model_alias' in grid_cell:
        alias = grid_cell['model_alias']
    em = adapt_map_to_model(em, cmp_id, str(alias), fbamodel)
    return em

In [66]:
em_list = []
alias_index = 1
maps_loaded = set()
for grid_cell in params['grid_maps']:
    maps_loaded.add(grid_cell['map_id'])
    em = build_grid_cell(grid_cell, escher_seed, api, str(alias_index))
    alias_index += 1
    em_list.append(em)

In [68]:
for em in em_list:
    em

In [71]:
#current_grid_layout =  get_shadow_grid_layout(map_data[1].canvas, build_config);
#draw_show_grid_layout(shadow_grid_layout, api);
#"iMM904;c;c;ModelSEED.Poly-beta-hydroxybutyrate Metabolism"
build_config = {
    'maps' : [
        'model;c;c;FBA Checked TCA Cycle.json'
    ],
    'x' : grid_size[0],
    'y' : grid_size[1],
}
build_config

{'maps': [], 'x': 4, 'y': 1}

In [ ]:
#build grid map
#cached maps
#cache shadow config
#cache structures

In [63]:
grid = modelseed_escher.EscherGrid()
grid_map = grid.build(em_list, grid_size)

In [70]:
grid_map.display_in_notebook()

In [ ]:
model_id = grid_block_data['model_id']

In [34]:
grid_cell['model_id']

'37534/89/1'

In [43]:
map_data.keys()

dict_keys(['layout', 'metadata'])

In [72]:
map_data = escher_seed.get_map('ModelSEED', 'ModelSEED', 'FBA Checked TCA Cycle')

In [74]:
with open('../data/html/data/map_model/base_map.json', 'w') as fh:
    fh.write(json.dumps(map_data.escher_map))

In [ ]:
ss